In [2]:
%matplotlib inline

import os
import glob
import numpy as np
import pandas as pd
from IPython.display import HTML
import networkx as nx

from numpy.fft import fft, ifft, fftfreq

from scipy.spatial.distance import pdist, squareform
from scipy.stats import rankdata, ttest_rel, ttest_1samp, pearsonr,spearmanr
from scipy.signal import hilbert

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.lines as mlinesÒ
import matplotlib.transforms as mtransforms
from matplotlib import gridspec
from matplotlib.animation import FuncAnimation

import nibabel as nib
from nilearn.input_data import NiftiLabelsMasker
from nilearn.plotting import plot_glass_brain, plot_stat_map, view_img, view_img_on_surf
from nilearn.image import new_img_like

from nltools.data import Brain_Data, Adjacency
from nltools.mask import roi_to_brain, expand_mask
from nltools.stats import isc, isfc
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm
from statsmodels.formula.api import mixedlm
from statsmodels.stats.multitest import multipletests

import warnings
warnings.filterwarnings('ignore')


In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import numpy as np

def regress_out(vec, control_matrices, upper_triangle_indices, standardize):
    """
    对给定的向量进行回归，剥离控制变量的影响，先标准化再回归（可选）。
    
    参数:
    - vec: np.array, 待剥离影响的向量
    - control_matrices: list of np.array, 控制变量的矩阵列表
    - upper_triangle_indices: tuple, 矩阵上三角的索引
    - standardize: bool, 是否先标准化数据，默认值为 True
    
    返回:
    - residuals: np.array, 回归后的残差
    """
    if control_matrices is None or len(control_matrices) == 0:
        return vec  # 如果没有控制变量，直接返回原始向量
    
    # 如果需要标准化，先标准化 vec
    if standardize:
        scaler = StandardScaler()
        vec_scaled = scaler.fit_transform(vec.reshape(-1, 1)).flatten()
    else:
        vec_scaled = vec
    
    # 收集控制变量的标准化版本（如果需要）
    control_vectors = []
    for control_matrix in control_matrices:
        control_vec = control_matrix[upper_triangle_indices]
        if standardize:
            control_vec_scaled = scaler.fit_transform(control_vec.reshape(-1, 1)).flatten()
        else:
            control_vec_scaled = control_vec
        control_vectors.append(control_vec_scaled)
    
    # 拼接控制变量矩阵
    control_matrix = np.column_stack(control_vectors)

    # 执行回归
    model = LinearRegression().fit(control_matrix, vec_scaled)
    
    # 计算残差
    residuals = vec_scaled - model.predict(control_matrix)
    
    return residuals

def mantel_with_multiple_controls(matrix1, matrix2, control_matrices=None, corr_type='spearman', permutations=1000, tail=2, standardize=True):
    """
    执行带有多个控制变量的 Mantel Test，包含置换检验。
    
    参数:
    - matrix1: np.array, 第一个距离矩阵 (NxN)
    - matrix2: np.array, 第二个距离矩阵 (NxN)
    - control_matrices: list of np.array, 控制变量的矩阵列表（默认为 None）
    - permutations: int, 置换次数
    - tail: int, p值的类型，1 表示单尾，2 表示双尾
    
    返回:
    - r_obs: 观察到的 Pearson 或 Spearman 相关系数
    - p_value: 置换检验的 p 值
    """
    # 确保输入矩阵是方阵
    assert matrix1.shape == matrix2.shape, "两个矩阵的形状必须相同"
    assert matrix1.shape[0] == matrix1.shape[1], "输入必须是方阵"
    
    # 提取上三角部分（不包括对角线）
    upper_triangle_indices = np.triu_indices_from(matrix1, k=1)
    vec1 = matrix1[upper_triangle_indices]
    vec2 = matrix2[upper_triangle_indices]
    
    # 对每个控制矩阵进行回归，剥离控制变量的影响
    residuals1 = regress_out(vec1, control_matrices, upper_triangle_indices,standardize=standardize)
    residuals2 = regress_out(vec2, control_matrices, upper_triangle_indices,standardize=standardize)
    
    # 计算观察到的相关系数
    if corr_type == 'pearson':
        r_obs, _ = pearsonr(residuals1, residuals2)
    elif corr_type == 'spearman':
        r_obs, _ = spearmanr(residuals1, residuals2)
    
    # 进行置换检验
    permuted_r = []
    n = matrix1.shape[0]
    
    for _ in range(permutations):
        # 随机打乱行列索引
        perm_indices = np.random.permutation(n)
        
        # 重新排列矩阵
        perm_matrix2 = matrix2[np.ix_(perm_indices, perm_indices)]
        
        # 对每个控制变量矩阵也进行相同的置换（如果存在控制变量）
        perm_control_matrices = []
        if control_matrices is not None:
            for control_matrix in control_matrices:
                perm_control_matrix = control_matrix[np.ix_(perm_indices, perm_indices)]
                perm_control_matrices.append(perm_control_matrix)
        
        # 提取置换后的上三角部分
        perm_vec2 = perm_matrix2[upper_triangle_indices]
        
        # 对置换后的矩阵进行回归，剥离控制变量的影响
        perm_residuals2 = regress_out(perm_vec2, perm_control_matrices, upper_triangle_indices, standardize=standardize)
        
        # 计算置换后的相关系数
        if corr_type == 'pearson':
            r_perm, _ = pearsonr(residuals1, perm_residuals2)
        elif corr_type == 'spearman':
            r_perm, _ = spearmanr(residuals1, perm_residuals2)
        
        permuted_r.append(r_perm)
    
    # 计算双尾或单尾 p 值
    p_value = _calc_pvalue(np.array(permuted_r), r_obs, tail)
    
    return r_obs, p_value

def _calc_pvalue(all_p, stat, tail):
    """计算基于置换分布的 p 值
    
    参数：
    - all_p: 置换分布的相关系数列表
    - stat: 观察到的统计量（如统计结果中的相关系数）
    - tail: (int) 1 或 2，表示单尾或双尾 p 值
    
    返回：
    - p_value: 计算得到的 p 值
    """
    denom = float(len(all_p)) + 1
    if tail == 1:
        numer = np.sum(all_p >= stat) + 1 if stat >= 0 else np.sum(all_p <= stat) + 1
    elif tail == 2:
        numer = np.sum(np.abs(all_p) >= np.abs(stat)) + 1
    else:
        raise ValueError("tail 必须是 1 或 2")
    return numer / denom


In [6]:

base_dir = '/Users/li/Desktop/debate/braindata'

sub_list = [f'sub-{x:0>3d}' for x in range(13,51)]
sub_list.remove('sub-021')

subs_roi_data = []
for sub in sub_list:
    csv_file = f'/Volumes/Li/task-debate/braindata/denoised 5/parcel data/Schaefer 200 combine 6 runs/{sub}_combined_time-series_Schaefer2018_200Parcels_7Networks.csv'
    sub_data = pd.read_csv(csv_file)
    subs_roi_data.append(sub_data.values)

all_brain_data = np.array(subs_roi_data)

mask_file = '/Users/li/Desktop/template/Schaefer/tpl-MNI152NLin2009cAsym_res-02_atlas-Schaefer2018_desc-200Parcels7Networks_dseg.nii.gz'
mask_img = nib.load(mask_file)
mask_data = mask_img.get_fdata()

nw_labels = pd.read_csv('/Users/li/Desktop/template/Schaefer/Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.Centroid_RAS.csv')
roi_name = list(nw_labels['ROI Name'])


In [8]:
bahav_data_dir = '/Users/li/Desktop/task-debate/behavdata'

sub_list_num = list(range(13,51))
sub_list_num.remove(21)

time_points = list(range(0,3000,60)) + [2986]  # every 1 minute
# time_points = list(range(0,3000,120)) + [2986] # every 2 min
# time_points = list(range(0,3000,300)) + [2986] # every 5 min
# time_points = [0, 252, 500, 772, 1098, 1484, 1892, 2464, 2986] # every speaker
# time_points = list(range(0,2987,2)) # every TR

# time_points = [0,80,168,208,252, 
#                326,364,464,500,
#                538,588,686,772,
#                860,986,1026,1098,
#                1204,1250,1406,1484,
#                1578,1722,1810,1892,
#                1972,2114,2216,2464,
#                2628,2756,2986] 

all_subject_data = []
for sub in sub_list_num:
    file_path = os.path.join(bahav_data_dir, 'during_scan', 'combined_6runs_per_TR_filter', f'subject_{sub}_TR_rate.csv')
    
    df = pd.read_csv(file_path)
    sub_data = df[df['time'].isin(time_points)]
    all_subject_data.append(list(sub_data['rate']))
    
attitude = pd.DataFrame(all_subject_data)

start_attitude = pd.DataFrame(attitude)[0]
start_attitude_SM = -np.abs(start_attitude.values[:, np.newaxis] - start_attitude.values)

attitude_change = attitude.diff(axis=1)
attitude_change = attitude_change.drop(attitude_change.columns[0], axis=1)
print(attitude_change.shape)

attitude_change_distances = -pdist(attitude_change)
# attitude_change_distances = -np.sqrt(pdist(attitude_change))
# attitude_change_distances = -np.log(pdist(attitude_change))
attitude_change_SM = squareform(attitude_change_distances)
print(attitude_change_SM.shape)

# sns.heatmap(attitude_change_SM)


(37, 50)
(37, 37)


In [10]:

subjects = list(range(13,51))
subjects.remove(21)

personality = pd.read_csv('/Users/li/Desktop/task-debate/behavdata/questionire_data/personality.csv')

selected_data = personality[personality['sub'].isin(subjects)]
selected_data = selected_data.set_index('sub').loc[subjects]

ages = selected_data['age'].values
age_diff_matrix = np.abs(ages[:, np.newaxis] - ages)

sex = selected_data['sex'].values
sex_diff_matrix = np.abs(sex[:, np.newaxis] - sex)

IUS = selected_data['IUS'].values
joint_IUS = (IUS[:, np.newaxis] + IUS)/2



In [12]:
subs_dISFC = np.load('/Users/li/Desktop/debate2025/results/subs_dISFC_seed89.npy')
subs_dISFC = np.arctanh(subs_dISFC)
print(subs_dISFC.shape)

(37, 199, 1464)


In [14]:
nw_labels = pd.read_csv('/Users/li/Desktop/template/Schaefer/Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.Centroid_RAS.csv')
nw_labels = nw_labels[nw_labels["ROI Label"] != 90].reset_index(drop=True)

vis, sm, da, va, limb, cont, default = [],[],[],[],[],[],[]
for i in range(199):
    net_name = nw_labels['ROI Name'][i].split('_')[2] 
    if net_name == 'Vis':
        vis.append(i)
    elif net_name == 'SomMot':
        sm.append(i)
    elif net_name == 'DorsAttn':
        da.append(i)
    elif net_name == 'SalVentAttn':
        va.append(i)
    elif net_name == 'Limbic':
        limb.append(i)
    elif net_name == 'Cont':
        cont.append(i)
    elif net_name == 'Default':
        default.append(i)
        
nets = [vis, sm, da, va, limb, cont, default]
nets_name = ['vis', 'sm', 'da', 'va', 'limb', 'cont', 'default']
print(len(default),default) 

45 [73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198]


In [18]:
subs_dISFC = np.load('/Users/li/Desktop/debate2025/results/subs_dISFC_seed89.npy')

intersub_dISFC_similarity = []
for edge in range(subs_dISFC.shape[1]):
    intersub_dISFC_similarity .append(Adjacency(1 - pairwise_distances(subs_dISFC[:, edge, :], metric='correlation'), matrix_type='similarity'))
brain_ISC = Adjacency(intersub_dISFC_similarity )

brain_ISC_np = np.array(brain_ISC.squareform())
print(brain_ISC_np.shape)

brain_ISC_Z_np = np.arctanh(brain_ISC_np)


(199, 37, 37)


In [20]:
nw_labels = pd.read_csv('/Users/li/Desktop/template/Schaefer/Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.Centroid_RAS.csv')
roi_name = list(nw_labels['ROI Name'])
roi_name.remove(roi_name[89])
print(len(roi_name))

# 按照组别分组 (例如: Vis)
groups = {}
for roi in roi_name:
    group_name = roi.split('_')[2]  # 假设组别是第三个部分，格式为: 7Networks_LH_Vis_1
    if group_name not in groups:
        groups[group_name] = []
    groups[group_name].append(roi)

199


In [22]:
nw_labels = pd.read_csv('/Users/li/Desktop/template/Schaefer/Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.Centroid_RAS.csv')
nw_labels = nw_labels[nw_labels["ROI Label"] != 90].reset_index(drop=True)

vis, sm, da, va, limb, cont, default = [],[],[],[],[],[],[]
for i in range(199):
    net_name = nw_labels['ROI Name'][i].split('_')[2] 
    if net_name == 'Vis':
        vis.append(i)
    elif net_name == 'SomMot':
        sm.append(i)
    elif net_name == 'DorsAttn':
        da.append(i)
    elif net_name == 'SalVentAttn':
        va.append(i)
    elif net_name == 'Limbic':
        limb.append(i)
    elif net_name == 'Cont':
        cont.append(i)
    elif net_name == 'Default':
        default.append(i)
        
nets = [vis, sm, da, va, limb, cont, default]
nets_name = ['vis', 'sm', 'da', 'va', 'limb', 'cont', 'default']
    

In [30]:
rlist1, plist1 = [],[]
for a,n in enumerate(nets):
    net = n
    subs_default_dISFC = subs_dISFC[:,net,:]
    
    distance_matrices = np.zeros((37, len(net), len(net)))
    for i in range(37):
        subject_data = subs_default_dISFC[i]  # (1464, 45)
        corr_matrix = np.corrcoef(subject_data)  # 皮尔逊相关系数
        distance_matrices[i] = np.arctanh(corr_matrix)  
    
    # ① 提取上三角部分
    n_subjects, n_nodes, _ = distance_matrices.shape 
    triu_indices = np.triu_indices(n_nodes, k=1)  # 获取上三角索引（不含对角线）
    
    # 将每个被试的距离矩阵转换为 1D 向量，形成 (37, N) 形状的矩阵
    vectorized_data = np.array([distance_matrices[i][triu_indices] for i in range(n_subjects)])  # (37, N)
    
    subs_sm = np.arctanh(np.corrcoef(vectorized_data))
    
    r, p = mantel_with_multiple_controls(attitude_change_SM, subs_sm,
                                            standardize=True, corr_type='pearson',
                                            # control_matrices = [start_attitude_SM, sex_diff_matrix, age_diff_matrix], 
                                            permutations=10000,tail=2)
    print(nets_name[a], r, p )
    rlist1.append(r)
    plist1.append(p)


vis 0.10665237882235096 0.32156784321567844
sm -0.15408580884337958 0.23367663233676633
da 0.08862309348271274 0.41855814418558146
va -0.009095499622260118 0.9347065293470653
limb 0.03331369591429557 0.785921407859214
cont -0.02888493713452425 0.811918808119188
default 0.2832390375608955 0.016198380161983803


In [25]:
rlist2, plist2 = [],[]
for a,n in enumerate(nets):
    net = n
    subs_default_dISFC = subs_dISFC[:,net,:]
    
    distance_matrices = np.zeros((37, len(net), len(net)))
    for i in range(37):
        subject_data = subs_default_dISFC[i]  # (1464, 45)
        corr_matrix = np.corrcoef(subject_data)  # 皮尔逊相关系数
        distance_matrices[i] = np.arctanh(corr_matrix)  
    
    # ① 提取上三角部分
    n_subjects, n_nodes, _ = distance_matrices.shape 
    triu_indices = np.triu_indices(n_nodes, k=1)  # 获取上三角索引（不含对角线）
    
    # 将每个被试的距离矩阵转换为 1D 向量，形成 (37, N) 形状的矩阵
    vectorized_data = np.array([distance_matrices[i][triu_indices] for i in range(n_subjects)])  # (37, N)
    
    subs_sm = np.arctanh(np.corrcoef(vectorized_data))
    
    r, p = mantel_with_multiple_controls(attitude_change_SM, subs_sm,
                                            standardize=True, corr_type='pearson',
                                            control_matrices = [start_attitude_SM, sex_diff_matrix, age_diff_matrix], 
                                            permutations=10000,tail=2)
    print(nets_name[a], r, p )
    rlist2.append(r)
    plist2.append(p)

vis 0.10397002851589929 0.3397660233976602
sm -0.13535027169114788 0.27377262273772623
da 0.09075160390436925 0.40875912408759124
va -0.0020384977842398157 0.9847015298470153
limb 0.04232038544069664 0.7316268373162683
cont -0.032680149369653785 0.7887211278872113
default 0.28592508106382764 0.015598440155984402


In [26]:
rlist3, plist3 = [],[]
for a,n in enumerate(nets):
    net = n
    subs_default_dISFC = subs_dISFC[:,net,:]
    
    distance_matrices = np.zeros((37, len(net), len(net)))
    for i in range(37):
        subject_data = subs_default_dISFC[i]  # (1464, 45)
        corr_matrix = np.corrcoef(subject_data)  # 皮尔逊相关系数
        distance_matrices[i] = np.arctanh(corr_matrix)  
    
    # ① 提取上三角部分
    n_subjects, n_nodes, _ = distance_matrices.shape 
    triu_indices = np.triu_indices(n_nodes, k=1)  # 获取上三角索引（不含对角线）
    
    # 将每个被试的距离矩阵转换为 1D 向量，形成 (37, N) 形状的矩阵
    vectorized_data = np.array([distance_matrices[i][triu_indices] for i in range(n_subjects)])  # (37, N)
    
    subs_sm = np.arctanh(np.corrcoef(vectorized_data))
    
    r, p = mantel_with_multiple_controls(attitude_change_SM, subs_sm,
                                            standardize=True, corr_type='spearman',
                                            # control_matrices = [start_attitude_SM, sex_diff_matrix, age_diff_matrix], 
                                            permutations=10000,tail=2)
    print(nets_name[a], r, p )
    rlist3.append(r)
    plist3.append(p)

vis 0.10470903691234097 0.32196780321967805
sm -0.13300464725863512 0.3046695330466953
da 0.1170177596436199 0.2676732326767323
va 0.012696964892859942 0.9099090090990901
limb 0.054386889189845924 0.6498350164983502
cont -0.015602606279085978 0.8926107389261074
default 0.21637314148573858 0.0628937106289371


In [27]:
rlist4, plist4 = [],[]
for a,n in enumerate(nets):
    net = n
    subs_default_dISFC = subs_dISFC[:,net,:]
    
    distance_matrices = np.zeros((37, len(net), len(net)))
    for i in range(37):
        subject_data = subs_default_dISFC[i]  # (1464, 45)
        corr_matrix = np.corrcoef(subject_data)  # 皮尔逊相关系数
        distance_matrices[i] = np.arctanh(corr_matrix)  
    
    # ① 提取上三角部分
    n_subjects, n_nodes, _ = distance_matrices.shape 
    triu_indices = np.triu_indices(n_nodes, k=1)  # 获取上三角索引（不含对角线）
    
    # 将每个被试的距离矩阵转换为 1D 向量，形成 (37, N) 形状的矩阵
    vectorized_data = np.array([distance_matrices[i][triu_indices] for i in range(n_subjects)])  # (37, N)
    
    subs_sm = np.arctanh(np.corrcoef(vectorized_data))
    
    r, p = mantel_with_multiple_controls(attitude_change_SM, subs_sm,
                                            standardize=True, corr_type='spearman',
                                            control_matrices = [start_attitude_SM, sex_diff_matrix, age_diff_matrix], 
                                            permutations=10000,tail=2)
    print(nets_name[a], r, p )
    rlist4.append(r)
    plist4.append(p)

vis 0.09896878002271584 0.3474652534746525
sm -0.12091580641962153 0.3247675232476752
da 0.11637499541227232 0.27247275272472754
va 0.012220898243566572 0.9085091490850915
limb 0.05666393491552367 0.6338366163383662
cont -0.02450754293774388 0.8368163183681632
default 0.21704424634571787 0.05649435056494351


In [31]:
result_data = {
    'pearson no-control': rlist1,
    'p1': plist1,
    'pearson control': rlist2,
    'p2': plist2,
    'spearman no-control': rlist3,
    'p3': plist3,
    'spearman control': rlist4,
    'p4': plist4
}

result_data_df = pd.DataFrame(result_data)

result_data_df.to_csv('/Users/li/Desktop/debate2025/results/ISFC-ISRSA-Exp1-seed89-1min-network-level-dp.csv', index=False)